In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt

### 모델이 각 epoch 마다 처리되는 상황을 보여줌
# 설치하기 : conda install -c conda-forge tqdm
from tqdm import tqdm

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist, fashion_mnist
from tensorflow.keras.optimizers import Adam
from keras import initializers

In [2]:
### Keras 가 Tensorflow 를 벡엔드로 사용할 수 있도록 설정
os.environ["KERAS_BACKEND"] = "tensorflow"

### 실험을 재현하고 동일한 결과를 얻을 수 있는지 
#   확인하기 위해 seed 를 설정합니다.
np.random.seed(10)

### 가짜 이미지 생성에 사용할 
#   랜덤 노이즈 벡터의 차원값을 설정합니다.
#   이후, 가짜 이미지 생성시 사용됨
random_dim = 100

In [10]:
def load_fashion_mnist_data():
    # 데이터를 로드합니다.
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    
    # 데이터를 -1 ~ 1 사이 값으로 normalize(정규화) 합니다.
    # 127.5 = 255.0 / 2 
    x_train = (x_train.astype(np.float32) - 127.5)/127.5

    # x_train 의 shape 를 (60000, 28, 28) 에서 (60000, 784) 로 바꿉니다.
    # 따라서 우리는 한 row 당 784 columns 을 가지게 됩니다.
    x_train = x_train.reshape(60000, 784)
    return (x_train, y_train, x_test, y_test)

In [11]:
# Adam Optimizer를 사용합니다.
"""
[옵티마이저 설정값]
- learning_rate : 0보다 크거나 같은 float 값. 학습률.
- beta_1: 0보다 크고 1보다 작은 float 값. 
          일반적으로 1에 가깝게 설정됩니다.
          최적화 속성으로 방향과 보폭 모두에 영향을 미침
- beta_2: 0보다 크고 1보다 작은 float 값. 
          일반적으로 1에 가깝게 설정됩니다.
          최적화 속성으로 방향과 보폭 모두에 영향을 미침
- epsilon: 0보다 크거나 같은 float형 fuzz factor. 
           None인 경우 K.epsilon()이 사용됩니다.
- decay: 0보다 크거나 같은 float 값. 
         업데이트마다 적용되는 학습률의 감소율입니다.
- amsgrad: 불리언. Adam의 변형인 AMSGrad의 적용 여부를 설정
"""
def get_optimizer():
    return Adam(learning_rate=0.0002, beta_1=0.5)

In [12]:
def get_generator(optimizer):
    generator = Sequential()
    ### stddev
    # - 파이썬 스칼라 혹은 스칼라 텐서. 생성할 난수(랜덤)값의 표준편차
    generator.add(Dense(256, input_dim=random_dim, 
            kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(512))
    ### 0의 값이 다음 계층으로 넘어가지 않거나,
    # - 넘기더라도 모든 뉴런의 출력값이 0이된다.
    # - 이러한 현상을 막기위해 개선된 계층이 LeakyReLU로
    # - 음수 값으로 변환된 값을 일부 넘길 수 있도록 처리한다.
    # - 음수 값의 허용범위를 0.2(20%)로 설정
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    
    ### tanh : 탄젠트(-1 ~ 1 사이의 값 사용)
    generator.add(Dense(784, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return generator

In [17]:
def get_discriminator(optimizer):
    discriminator = Sequential()
    ### stddev
    # - 파이썬 스칼라 혹은 스칼라 텐서. 생성할 난수(랜덤)값의 표준편차
    discriminator.add(Dense(1024, input_dim=784, 
            kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', 
                          optimizer=optimizer)
    
    return discriminator

In [13]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # 우리는 Generator와 Discriminator를 동시에 학습시키고 싶을 때 
    # trainable을 False로 설정합니다.
    discriminator.trainable = False

    # GAN 입력 (노이즈)은 위에서 100 차원으로 설정했습니다.
    gan_input = Input(shape=(random_dim, ))

    # Generator의 결과는 이미지 입니다.
    x = generator(gan_input)

    # Discriminator의 결과는 이미지가 진짜인지 가짜인지에 대한 확률입니다.
    gan_output = discriminator(x)
    
    ### model() : 입력이 2개인 Multi(input, output) 모델 생성 방법임
    # - model.add() 함수는 1개의 입력만 처리할 수 있음
    # - 입력이 2개인 layer(계층)을 사용하고자 할 때는 
    #   멀티모델(multi model) 사용
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return gan

In [14]:
def plot_generated_images(epoch, generator, 
                          examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], 
                   interpolation='nearest', cmap='gray_r')
        plt.axis('off')
        
    plt.tight_layout()
    plt.savefig('./03_images/gan_generated_image_epoch_%d.png' % epoch)

In [25]:
def train(epochs=400, batch_size=128):
    # train 데이터와 test 데이터를 가져옵니다.
    x_train, y_train, x_test, y_test = load_fashion_mnist_data()

    ### train 데이터를 128 사이즈의 batch 로 나눕니다.
    # batch_size : 몇 개의 샘플로 가중치를 갱신할 것인지 설정
    batch_count = x_train.shape[0] // batch_size

    ### GAN 네트워크를 만듭니다.
    # - 옵티마이저 함수 호출
    adam = get_optimizer()
    # - 생성기 함수 호출
    generator = get_generator(adam)
    # - 판별기 함수 호출
    discriminator = get_discriminator(adam)
    # - 생성기와 판별기를 모두 합친 함수 호출
    gan = get_gan_network(discriminator, random_dim, 
                          generator, adam)
    
    ### 훈련 시작하기..
    for e in range(1, epochs+1):
        print('-'*15, 'Epoch {}'.format(e), '-'*15)
        for _ in tqdm(range(batch_count)):
            # 입력으로 사용할 random 노이즈와 이미지를 가져옵니다.
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])

            image_batch = x_train[np.random.randint(0, 
                                                    x_train.shape[0], 
                                                    size=batch_size)]

            # MNIST 이미지를 생성합니다.
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])
            
            ### np.zeros() : 0으로 채워진 1차원 배열 생성
            y_dis = np.zeros(2*batch_size)
            y_dis[ : batch_size] = 0.9

            ### Discriminator를 학습시킵니다.
            ### trainable = True
            # - classification layer를 학습 한 뒤에는 
            #   이제 feature map과 classification layer를 모두 포함한 
            #  전체 layer를 다시 학습합니다. 
            #  그래서 model.layers[i].trainable = True로 변경합니다.
            # - 보통 이런식으로 학습하면 처음부터 
            #   feature map layer를 random값으로 시작하지 않고 
            #   imagenet pretrained된 weight값을 이용할 수 있으므로 
            #   선호되는 방식임
            discriminator.trainable = True
            
            ### train_on_batch()
            # - train_on_batch는 이미 존재하는 모델에 
            #   새로운 학습 데이터 셋을 받아서 
            #   바로바로 이전의 모델로 학습시킬 수 있는 장점이 있음
            # - GAN은 기존 이미지의 가중치 값을 그대로 이용해서 향상시키기 때문
            discriminator.train_on_batch(X, y_dis)

            # Generator를 학습시킵니다.
            noise = np.random.normal(0, 1, 
                                     size=[batch_size, random_dim])
            
            ### ones() : 1로 가득찬 배열을 생성
            # - zeros()의 반대 개념
            y_gen = np.ones(batch_size)
            
            ### trainable = = False
            # - classification layer를 학습하면 해당 모델은 사용 안함
            # - trainable = True의 반대 개념
            discriminator.trainable = False
            
            ### train_on_batch()
            # - train_on_batch는 이미 존재하는 모델에 
            #   새로운 학습 데이터 셋을 받아서 
            #   바로바로 이전의 모델로 학습시킬 수 있는 장점이 있음
            # - GAN은 기존 이미지의 가중치 값을 그대로 이용해서 향상시키기 때문
            gan.train_on_batch(noise, y_gen)
        
        ### epoch가 1 또는 20번째 마다 시각화 함수 호출(저장 시킴)
        if e == 1 or e % 5 == 0:
            plot_generated_images(e, generator)

In [26]:
train()

--------------- Epoch 1 ---------------


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 12.22it/s]


--------------- Epoch 2 ---------------


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:41<00:00, 11.26it/s]


--------------- Epoch 3 ---------------


...

100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 12.11it/s]


--------------- Epoch 61 ---------------


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 12.01it/s]


--------------- Epoch 62 ---------------


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]